## Simulation using Gaussian distributed volumes

In [1]:
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt

%cd utils
from sim_functions import optimize_measurement
%cd ..

c:\Users\User\msc_project\Image-processing-strategies\MRsim\utils
c:\Users\User\msc_project\Image-processing-strategies\MRsim


c:\Users\User\miniconda3\envs\mrsim\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Edit the echo time (or list of echo times) and the flip angle:
TE_list = [0.008, 0.0095]#, 0.011, 0.0125, 0.014, 0.0155]
te_secs = [8, 9.5] # Using it in seconds to make it easier to run with texture on T2s values
flip_ang = 24
# The last requirement for the code is the dimensions, which can be easily acquired from the volume you work with
# The dimensions needed because we will create a 4D array, where we will store the echo time data and we want to maintain the initial 3D dimensions.
data = nib.load("data/ratatouille_crop.nii.gz")
np_data = data.get_fdata()
dimensions = np.array(np_data.shape)
dimensions

array([109, 129, 218])

In [4]:
# Loading the data, Gaussian distributed volumes
g_pd= nib.load("data/gauss_pd_corr_dist.nii.gz")
gauss_pd_vol = g_pd.get_fdata()
g_t2s = nib.load("data/gauss_t2s_corr_dist.nii.gz")
gauss_t2s_vol = g_t2s.get_fdata()
#deltaB0 = nib.load("data/fieldmap_crop.nii.gz")
pro_deltaB0 = nib.load("output/realistic_fieldmap.nii.gz")
dB0 = pro_deltaB0.get_fdata()
fieldStrength = 3
# suppress warnings due to Overflow on exponential while iterating
#import warnings
#warnings.filterwarnings('ignore') 

In [5]:
# I think the problem is when we have T2s value sooooooooo low, like in the lungs and air where we have 0.01
decay_gauss = np.zeros(dimensions)
for i in range(dimensions[0]):
    for j in range(dimensions[1]):
        for k in range(dimensions[2]):
            with np.errstate(over='raise'):
                try:
                    decay_gauss[i, j, k] = np.exp(-0.008 / gauss_t2s_vol[i, j, k])
                except FloatingPointError:
                    print(f"Overflow detected at index ({i}, {j}, {k}) with T2* value {gauss_t2s_vol[i, j, k]}")
                    decay_gauss[i, j, k] = np.finfo(np.float64).max  # Assign max value or another placeholder

In [30]:
#An example of what a values can rise a overflow in exponential
# This value was in the T2s volume because the gaussian distribution centered at 0.01
# Which was 'air' label created a few pixels with this offlayer values that caused the issue 
# returning infinity and hence ITK could not open the image
#np.exp(-0.008/-4.346339950507538e-06)

C:\Users\User\AppData\Local\Temp\ipykernel_83356\1727653921.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(-0.008/-4.346339950507538e-06)


inf

In [5]:
# For gaussian data
gauss_magnitude, gauss_phase = optimize_measurement(gauss_pd_vol, gauss_t2s_vol, dimensions, dB0, flip_ang, TE_list, fieldStrength)
# By default, B0 should be 3 Tesla

Starting optimize_measurement
Processing TE[0] = 0.008
Starting optimized_signal
sin:  0.4067366430758002
handedness=left
Coefficient of phase factor:  -6.420532498559999j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[1] = 0.0095
Starting optimized_signal
sin:  0.4067366430758002
handedness=left
Coefficient of phase factor:  -7.624382342039999j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Finished optimize_measurement


In [6]:
# Finally we save the magnitude and the phase to analyze them
# Remember to change the name, if not it will overwrite it!
# This version for gaussian distributed volumes
temp_mag_g = nib.Nifti1Image(gauss_magnitude, affine=data.affine)
temp_ph_g = nib.Nifti1Image(gauss_phase, affine=data.affine)
path_mag_g = os.path.join('output','gauss_sim_mag.nii.gz')
path_ph_g = os.path.join('output','gauss_sim_phase.nii.gz')
nib.save(temp_mag_g, path_mag_g)
nib.save(temp_ph_g, path_ph_g)